In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
import pandas as pd
from skimage import io
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image

In [39]:
class CatsAndDogsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        label = torch.tensor(int(self.annotations.iloc[index,1]))
        if self.transform:
            image = self.transform(image)
        return (image, label)


In [51]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

in_channel = 3
num_classes = 2
learning_rate = 1e-3
num_epochs = 1

In [41]:
dataset = CatsAndDogsDataset(csv_file = 'Data/dogs-vs-cats/dogs-vs-cats-index.csv', root_dir = 'Data/dogs-vs-cats/data', transform=transforms.ToTensor())
print(len(dataset))

24997


In [42]:
train_set, test_set = torch.utils.data.random_split(dataset, [24000, 997])
batch_size = 100
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [43]:
model = torchvision.models.googlenet(pretrained=True)
model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

In [44]:
print(model)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [48]:
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Sequential(
    nn.Linear(1024, 1000),
    nn.Linear(1000, 2),
)

In [52]:
for epoch in range(num_epochs):
    losses = []
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        scores = model(data)
        loss = loss_function(scores, targets)
        losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    
        print(f'Cost at batch {batch_idx+1} is {sum(losses)/len(losses):.5f}')
    print(f'Cost at epoch {epoch+1} is {sum(losses)/len(losses):.5f}')
    print()
print('Training Completed!')

Cost at batch 1 is 0.67153
Cost at batch 2 is 0.67695
Cost at batch 3 is 0.67794
Cost at batch 4 is 0.67700
Cost at batch 5 is 0.67785
Cost at batch 6 is 0.67968
Cost at batch 7 is 0.68117
Cost at batch 8 is 0.68085
Cost at batch 9 is 0.68251
Cost at batch 10 is 0.68316
Cost at batch 11 is 0.68365
Cost at batch 12 is 0.68505
Cost at batch 13 is 0.68414
Cost at batch 14 is 0.68255
Cost at batch 15 is 0.68146


KeyboardInterrupt: 

In [ ]:
# Model Testing
def check_accuracy(loader, model):
    model.eval() # Set model into evaluation mode!
    num_samples = 0
    num_correct = 0
    for data, targets in loader:
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        predicted_output = model(data)
        
        _, prediction = predicted_output.max(1)
        num_samples += predicted_output.size(0)
        num_correct += (prediction==targets).sum()

    print(f'Correctly identified samples: {num_correct}')
    print(f'Total samples: {num_samples}')
    print(f'The Validation Accuracy is {num_correct / num_samples * 100.00:.2f}')
    model.train() # Setting model back to training data

In [ ]:
print("Checking Accuracy on Training Data...")
check_accuracy(train_loader, model)
print()
print("Checking Accuracy on Testing Data...")
check_accuracy(test_loader, model)